# Final Project Submission
Please fill out:

-  Student name: Steven Rosa
-  Student pace: part time
-  Scheduled project review date/time: Mon 4 Feb 2019 2pm EST
-  Instructor name: Jeff Herman
-  Blog post URL:

# Northwind Data Analysis

In [1]:
#Import libraries
import sqlalchemy #ORM
from sqlalchemy import create_engine #To connect to the database
from sqlalchemy.orm import Session, sessionmaker #To interact with the database as object
from sqlalchemy import inspect #To look at tables and column names
from sqlalchemy import MetaData #to reflect the engine object 
from sqlalchemy.ext.automap import automap_base # to create mappings from the reflection
import pandas as pd #To read in parts of the database as DataFrames

In [ ]:
#Write functions

In [2]:
#Instantiate a session of the ORM
#Connect to the sqlite database file
engine = create_engine('sqlite:///Northwind_small.sqlite', echo = True)
#Bind a session object to the engine
Session = sessionmaker(bind=engine)
#Instantiate the session
session = Session()

In [4]:
#An object to get a first look at the database
inspector = inspect(engine)

2019-01-28 19:52:38,657 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2019-01-28 19:52:38,658 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 19:52:38,659 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2019-01-28 19:52:38,660 INFO sqlalchemy.engine.base.Engine ()


In [14]:
#Make a dict of list of names of all the columns in all the tables
#for easy reference

#Get list of all table names
table_names = inspector.get_table_names()

tables_columns = dict()
#For each table name
for table in table_names:
    column_names = []
    #for each column in the table, get just the name
    for column in inspector.get_columns(table):
        column_names.append(column['name'])
    tables_columns[table] = column_names

In [16]:
tables_columns['OrderDetail']

['Id', 'OrderId', 'ProductId', 'UnitPrice', 'Quantity', 'Discount']

In [53]:
#Pull raw results from the database
#conn = engine.connect()
#results = conn.execute('''SELECT *
#                            FROM Category
#                        ;''')
#print(results.fetchall())

In [52]:
###conn = engine.connect()
#results = conn.execute("SELECT * FROM [Order];")
#df = pd.DataFrame(results.fetchall())
#df.head()

In [54]:
#results = conn.execute('''SELECT *
#                            FROM OrderDetail
#                        ;''')
#df = pd.DataFrame(results.fetchall())
#df.head()

In [55]:
#df = pd.read_sql_query('''SELECT * 
#                           FROM [Order]
#                       ;''', engine)
#df.head()

In [47]:
#df = pd.read_sql_query("SELECT * FROM [Order] WHERE CUSTOMERId = 'VINET'", engine)
#df.head()

In [56]:
df = pd.read_sql_query("""SELECT OrderDetail.*, [Order].CustomerId 
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount != 0.0
                        """, engine)
df.head()

2019-01-28 13:38:29,447 INFO sqlalchemy.engine.base.Engine SELECT OrderDetail.*, [Order].CustomerId 
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount != 0.0
                        


INFO:sqlalchemy.engine.base.Engine:SELECT OrderDetail.*, [Order].CustomerId 
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount != 0.0
                        


2019-01-28 13:38:29,449 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


,Id,OrderId,ProductId,UnitPrice,Quantity,Discount,CustomerId
0,10250/51,10250,51,42.4,35,0.15,HANAR
1,10250/65,10250,65,16.8,15,0.15,HANAR
2,10251/22,10251,22,16.8,6,0.05,VICTE
3,10251/57,10251,57,15.6,15,0.05,VICTE
4,10252/20,10252,20,64.8,40,0.05,SUPRD


In [ ]:
#WHERE OrderDetail.Discount != 0.0

In [19]:
#Look at all orders with and without discounts
df = pd.read_sql_query("""SELECT OrderDetail.*, [Order].CustomerId
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            ORDER BY [Order].CustomerId DESC, [Order].Id DESC, OrderDetail.Discount DESC
                        """, engine)


2019-01-28 20:16:06,172 INFO sqlalchemy.engine.base.Engine SELECT OrderDetail.*, [Order].CustomerId
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            ORDER BY [Order].CustomerId DESC, [Order].Id DESC, OrderDetail.Discount DESC
                        
2019-01-28 20:16:06,172 INFO sqlalchemy.engine.base.Engine ()


In [20]:
#Look at all orders with and without discounts
df.head()

,Id,OrderId,ProductId,UnitPrice,Quantity,Discount,CustomerId
0,11044/62,11044,62,49.30,12,0.0,WOLZA
1,10998/24,10998,24,4.50,12,0.0,WOLZA
2,10998/61,10998,61,28.50,7,0.0,WOLZA
3,10998/74,10998,74,10.00,20,0.0,WOLZA
4,10998/75,10998,75,7.75,30,0.0,WOLZA


In [17]:
#Look at number of orders per cust id
df = pd.read_sql_query("""SELECT [Order].CustomerId, Count(*) orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        """, engine)

2019-01-28 20:14:11,979 INFO sqlalchemy.engine.base.Engine SELECT [Order].CustomerId, Count(*) orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        
2019-01-28 20:14:11,980 INFO sqlalchemy.engine.base.Engine ()


In [18]:
#Look at number of orders per cust id
df.head()

,CustomerId,orders
0,ALFKI,12
1,ANATR,10
2,ANTO,17
3,AROUT,30
4,BERGS,52


In [23]:
#Look at number of orders per customer w/o discount
df_wo_discount = pd.read_sql_query("""SELECT [Order].CustomerId, Count(*) orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount == 0.0
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        """, engine)

#Look at number of orders per customer w/ discount
df_w_discount = pd.read_sql_query("""SELECT [Order].CustomerId, Count(*) orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount != 0.0
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        """, engine)

2019-01-28 20:19:19,662 INFO sqlalchemy.engine.base.Engine SELECT [Order].CustomerId, Count(*) orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount == 0.0
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        
2019-01-28 20:19:19,663 INFO sqlalchemy.engine.base.Engine ()
2019-01-28 20:19:19,669 INFO sqlalchemy.engine.base.Engine SELECT [Order].CustomerId, Count(*) orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            WHERE OrderDetail.Discount != 0.0
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        
2019-01-28 20:19:19,670 INFO sqlalchemy.engine.base.Engine ()


In [24]:
df_wo_discount.head()

,CustomerId,orders
0,ALFKI,6
1,ANATR,10
2,ANTO,9
3,AROUT,23
4,BERGS,34


In [26]:
df_w_discount.head()

,CustomerId,orders
0,ALFKI,6
1,ANTO,8
2,AROUT,7
3,BERGS,18
4,BLONP,8


In [45]:
#SAMPLE CODE
#select ( SELECT COUNT(u1.user_id)
#           FROM users u1
#          WHERE u1.username LIKE '$user' ) as count_u
#     , ( SELECT COUNT(u2.user_id)
#           FROM users u2
#          WHERE u2.user_email LIKE '$email' ) as count_e

#Look at number of orders w and wo discount per cust id
df = pd.read_sql_query("""SELECT [Order].CustomerId, 
                                (SELECT Count(*) orders
                                    FROM OrderDetail 
                                    INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                                    WHERE OrderDetail.Discount == 0.0
                                    GROUP BY [Order].CustomerId
                                    ORDER BY [Order].CustomerId ASC) no_discount,    
                            Count(*) total_orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        """, engine)

####RESUME HERE 1/29. Inner query is only showing ALFKI no discount orders.

2019-01-28 20:44:32,243 INFO sqlalchemy.engine.base.Engine SELECT [Order].CustomerId, 
                                (SELECT Count(*) orders
                                    FROM OrderDetail 
                                    INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                                    WHERE OrderDetail.Discount == 0.0
                                    GROUP BY [Order].CustomerId
                                    ORDER BY [Order].CustomerId ASC) no_discount,    
                            Count(*) total_orders
                            FROM OrderDetail 
                            INNER JOIN [Order] on OrderDetail.OrderId = [Order].Id
                            GROUP BY [Order].CustomerId
                            ORDER BY [Order].CustomerId ASC
                        
2019-01-28 20:44:32,243 INFO sqlalchemy.engine.base.Engine ()


In [46]:
#	ALFKI	12
#1	ANATR	10
#2	ANTO	17
#3	AROUT	30
#4	BERGS	52
df.head()

,CustomerId,no_discount,total_orders
0,ALFKI,6,12
1,ANATR,6,10
2,ANTO,6,17
3,AROUT,6,30
4,BERGS,6,52


In [36]:
df.shape

(89, 3)

In [45]:
#Look at the levels of discount

df = pd.read_sql_query("""SELECT OrderDetail.*
                            FROM OrderDetail 
                        """, engine)
df['Discount'].value_counts()

2019-01-28 11:21:40,070 INFO sqlalchemy.engine.base.Engine SELECT OrderDetail.*
                            FROM OrderDetail 
                        


INFO:sqlalchemy.engine.base.Engine:SELECT OrderDetail.*
                            FROM OrderDetail 
                        


2019-01-28 11:21:40,073 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


0.00    1317
0.05     185
0.10     173
0.20     161
0.15     157
0.25     154
0.03       3
0.02       2
0.01       1
0.04       1
0.06       1
Name: Discount, dtype: int64

In [78]:
#Look at how many customers each company has
df = pd.read_sql_query("""SELECT Customer.Id, Customer.CompanyName
                            FROM Customer
                            ORDER BY Customer.CompanyName
                        """, engine)

2019-01-28 14:21:37,040 INFO sqlalchemy.engine.base.Engine SELECT Customer.Id, Customer.CompanyName
                            FROM Customer
                            ORDER BY Customer.CompanyName
                        


INFO:sqlalchemy.engine.base.Engine:SELECT Customer.Id, Customer.CompanyName
                            FROM Customer
                            ORDER BY Customer.CompanyName
                        


2019-01-28 14:21:37,042 INFO sqlalchemy.engine.base.Engine ()


INFO:sqlalchemy.engine.base.Engine:()


In [83]:
#Every company has only one customer. Orders can therefore be counted by customer or by company.
sum(df['CompanyName'].value_counts() > 1)

0

In [4]:
# JUST AN EXAMPLE
"""SELECT o.ID, c.CompanyName, Count(*) num_orders 
    FROM [Order] o
    INNER JOIN Customer c on o.CustomerID = c.ID 
    GROUP BY c.CompanyName 
    ORDER BY num_orders DESC"""

df = pd.read_sql_query("""SELECT o.ID, c.CompanyName, Count(*) num_orders FROM [Order] \
o INNER JOIN Customer c on o.CustomerID = c.ID GROUP BY c.CompanyName ORDER BY num_orders DESC""", engine)
df.head()

2019-01-28 10:39:55,275 INFO sqlalchemy.engine.base.Engine SELECT o.ID, c.CompanyName, Count(*) num_orders FROM [Order] o INNER JOIN Customer c on o.CustomerID = c.ID GROUP BY c.CompanyName ORDER BY num_orders DESC
2019-01-28 10:39:55,276 INFO sqlalchemy.engine.base.Engine ()


,Id,CompanyName,num_orders
0,11064,Save-a-lot Markets,31
1,11072,Ernst Handel,30
2,11021,QUICK-Stop,28
3,11050,Folk och fä HB,19
4,11063,Hungry Owl All-Night Grocers,19


In [70]:
#######################################
#AUTOMAPPER BITS
#Instantiate metadata to reflect the engine
#metadata = MetaData()
#metadata.reflect(engine)

In [4]:
#Create mappings to treat the database elements as OOP objects
Base = automap_base(metadata=metadata)
Base.prepare()

In [ ]:
#['Category', 'Customer', 'CustomerCustomerDemo', 'CustomerDemographic', 
#'Employee', 'EmployeeTerritory', 'Order', 'OrderDetail', 'Product', 'Region', 'Shipper', 'Supplier', 'Territory']

In [5]:
#Create an object for each table
Category = Base.classes.Category
Customer = Base.classes.Customer
CustomerCustomerDemo = Base.classes.CustomerCustomerDemo
CustomerDemographic = Base.classes.CustomerDemographic
Employee = Base.classes.Employee
EmployeeTerritory = Base.classes.EmployeeTerritory
Order = Base.classes.Order
OrderDetail = Base.classes.OrderDetail
Product = Base.classes.Product
Region = Base.classes.Region
Shipper = Base.classes.Shipper
Supplier = Base.classes.Supplier
Territory = Base.classes.Territory

In [6]:
categories = session.query(Category).all()

for cat in categories:
    print(cat.CategoryName, cat.Description)

2019-01-27 15:22:07,052 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2019-01-27 15:22:07,053 INFO sqlalchemy.engine.base.Engine SELECT "Category"."Id" AS "Category_Id", "Category"."CategoryName" AS "Category_CategoryName", "Category"."Description" AS "Category_Description" 
FROM "Category"
2019-01-27 15:22:07,054 INFO sqlalchemy.engine.base.Engine ()
Beverages Soft drinks, coffees, teas, beers, and ales
Condiments Sweet and savory sauces, relishes, spreads, and seasonings
Confections Desserts, candies, and sweet breads
Dairy Products Cheeses
Grains/Cereals Breads, crackers, pasta, and cereal
Meat/Poultry Prepared meats
Produce Dried fruit and bean curd
Seafood Seaweed and fish


In [9]:
for instance in session.query(Category):
    print(instance.Description)

2019-01-27 20:15:03,448 INFO sqlalchemy.engine.base.Engine SELECT "Category"."Id" AS "Category_Id", "Category"."CategoryName" AS "Category_CategoryName", "Category"."Description" AS "Category_Description" 
FROM "Category"
2019-01-27 20:15:03,451 INFO sqlalchemy.engine.base.Engine ()
Soft drinks, coffees, teas, beers, and ales
Sweet and savory sauces, relishes, spreads, and seasonings
Desserts, candies, and sweet breads
Cheeses
Breads, crackers, pasta, and cereal
Prepared meats
Dried fruit and bean curd
Seaweed and fish


In [10]:
countries = []
for instance in session.query(Customer):
    countries.append(instance.Country)

2019-01-27 20:15:19,574 INFO sqlalchemy.engine.base.Engine SELECT "Customer"."Id" AS "Customer_Id", "Customer"."CompanyName" AS "Customer_CompanyName", "Customer"."ContactName" AS "Customer_ContactName", "Customer"."ContactTitle" AS "Customer_ContactTitle", "Customer"."Address" AS "Customer_Address", "Customer"."City" AS "Customer_City", "Customer"."Region" AS "Customer_Region", "Customer"."PostalCode" AS "Customer_PostalCode", "Customer"."Country" AS "Customer_Country", "Customer"."Phone" AS "Customer_Phone", "Customer"."Fax" AS "Customer_Fax" 
FROM "Customer"
2019-01-27 20:15:19,575 INFO sqlalchemy.engine.base.Engine ()


In [11]:
countries_df = pd.DataFrame(countries)
countries_df[0].value_counts()

USA            13
France         11
Germany        11
Brazil          9
UK              7
Spain           5
Mexico          5
Venezuela       4
Argentina       3
Italy           3
Canada          3
Switzerland     2
Denmark         2
Austria         2
Sweden          2
Belgium         2
Portugal        2
Finland         2
Norway          1
Ireland         1
Poland          1
Name: 0, dtype: int64

In [12]:
regions = []
for instance in session.query(Customer):
    regions.append(instance.Region)

regions_df = pd.DataFrame(regions)
regions_df[0].value_counts()

2019-01-27 20:15:27,313 INFO sqlalchemy.engine.base.Engine SELECT "Customer"."Id" AS "Customer_Id", "Customer"."CompanyName" AS "Customer_CompanyName", "Customer"."ContactName" AS "Customer_ContactName", "Customer"."ContactTitle" AS "Customer_ContactTitle", "Customer"."Address" AS "Customer_Address", "Customer"."City" AS "Customer_City", "Customer"."Region" AS "Customer_Region", "Customer"."PostalCode" AS "Customer_PostalCode", "Customer"."Country" AS "Customer_Country", "Customer"."Phone" AS "Customer_Phone", "Customer"."Fax" AS "Customer_Fax" 
FROM "Customer"
2019-01-27 20:15:27,314 INFO sqlalchemy.engine.base.Engine ()


Western Europe     28
North America      16
South America      16
Southern Europe    10
British Isles       8
Central America     5
Northern Europe     4
Scandinavia         3
Eastern Europe      1
Name: 0, dtype: int64

In [ ]:
names = []
for instance in session.query(Customer):
    names.append(instance.ContactName)
    
names_df = pd.DataFrame(names)
names_df[0].value_counts()

In [ ]:
len(set(names))

In [ ]:
session.query(Customer).column_descriptions

In [18]:
result = session.execute('''SELECT *
                            FROM Customer
                        ;''')

2019-01-26 15:57:22,647 INFO sqlalchemy.engine.base.Engine SELECT *
                            FROM Customer
                        ;
2019-01-26 15:57:22,650 INFO sqlalchemy.engine.base.Engine ()


In [19]:
customer_df = pd.DataFrame(result.fetchall())

In [21]:
customer_df.shape

(91, 11)

## Hypothesis 1
Do discounts have a statistically significant effect on the number of products customers order? If so, at what level(s) of discount?

product (quantity) ordered by customers, no discount

product (quantity) order by customers, wiht discount

H0: Discount does not have an effect on the number of products ordered by a customer. Control = Treatment

H1: Discount has an effect on the number of products ordered by a customer. Control < or > Treament